In [1]:
from pyspark import SparkContext
from datetime import datetime

In [2]:
sc = SparkContext(appName="Assignment_conditions")

In [3]:
threshold=40

In [4]:
data = sc.textFile("testing.csv")
data = data.map(lambda line: line.split(",")).filter(lambda v: v[0]!="START")
condition_map = data.map(lambda line: (int(line[4]),line[5])).distinct()#reduceByKey(lambda val1, val2: val1)
condition_map.take(5)

[(65363002, 'Otitis media'),
 (444814009, 'Viral sinusitis (disorder)'),
 (233678006, 'Childhood asthma'),
 (232353008, 'Perennial allergic rhinitis with seasonal variation'),
 (446096008, 'Perennial allergic rhinitis')]

In [5]:
format_time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
condition_map.saveAsTextFile("{0}/{1}".format("condition_map", format_time))


In [6]:
patient_rdd2 = data.map(lambda line: (line[2],line[4])).groupByKey().map(lambda tp: [int(j) for j in set(tp[1])])#.map(lambda k: (k[0],[int[j] for j in k[1]]))
n_baskets = patient_rdd2.count()
patient_rdd2.take(50)

[[446096008, 283371005, 284551006, 444814009, 72892002, 195662009],
 [44465007, 307731004, 10509002, 444814009, 72892002, 195662009],
 [65966004, 55822004, 162864005, 24079001],
 [64859006, 70704007, 40055000, 59621000, 444814009, 162864005],
 [156073000,
  19169002,
  55822004,
  82423001,
  15777000,
  10509002,
  40055000,
  72892002,
  444814009,
  75498004],
 [195662009, 444814009, 283385000],
 [195662009, 65363002, 43878008],
 [44465007, 10509002, 444814009, 58150001, 43878008, 195662009],
 [446096008,
  44465007,
  284549007,
  72892002,
  198992004,
  53741008,
  444814009,
  43878008],
 [64859006, 162864005, 59621000],
 [444814009, 10509002, 70704007, 43878008],
 [241929008, 24079001, 444814009, 233678006, 43878008, 232353008],
 [70704007, 10509002, 128613002, 195662009, 703151001],
 [446096008, 444814009, 43878008, 62106007, 65363002],
 [126906006,
  92691004,
  15777000,
  68496003,
  40055000,
  59621000,
  271737000,
  162573006,
  403190006,
  444814009,
  162864005,
  37

In [7]:

frequent_singles = patient_rdd2.flatMap(lambda l: [(i,1) for i in l]).reduceByKey(lambda v1,v2:v1+v2).sortBy(lambda a:a[1],False).filter(lambda k: k[1]>threshold)
frequent_singles.take(10)

[(444814009, 95), (195662009, 69), (10509002, 58), (162864005, 43)]

In [8]:
freq_single_item_set = set([k[0] for k in frequent_singles.collect()])
print(freq_single_item_set)

{444814009, 10509002, 195662009, 162864005}


In [9]:
def pair_forming(list):
    pairs=[]
    list.sort()
    for i in range(0,len(list)):
        if list[i] in freq_single_item_set:
            for j in range(i+1,len(list)):
                if  list[i]!=list[j] and list[j] in freq_single_item_set:
                    pairs.append((list[i],list[j]))
    return pairs

In [10]:
def triple_forming(list):
    triples=[]
    pairs = pair_forming(list)
    for i,j in pairs:
        for k in list:
            if (i,j) in frequent_pairs and k in freq_single_item_set and k!=i and k!=j:#or (j,i) in frequent_pairs#prob unecessary
                  triples.append(tuple(sorted([i,j,k])))
    return triples

In [11]:
###COUNTING PAIRS
pair_count = patient_rdd2.flatMap(pair_forming).map(lambda p:(p,1)).reduceByKey(lambda v1,v2: v1+v2).filter(lambda tp:tp[1]>=threshold)
pair_count.sortBy(lambda a: a[1],False).take(10)

[((195662009, 444814009), 44)]

In [12]:
###FREQUENT PAIR SET
frequent_pairs = set(pair_count.map(lambda tp:tp[0]).collect())
print(frequent_pairs)

{(195662009, 444814009)}


In [13]:
###COUNTING TRIPLES
triple_count = patient_rdd2.flatMap(triple_forming).map(lambda p:(p,1)).reduceByKey(lambda v1,v2: v1+v2).filter(lambda tp:tp[1]>=threshold)
triple_count.sortBy(lambda a: a[1], False).take(10)

[]

In [14]:
###FREQUENT TRIPLE SET
frequent_triples = set(triple_count.map(lambda tp:tp[0]).collect())
print(frequent_triples)

set()


In [15]:
min_std_lift = 0.2
n_baskets

152

In [16]:
def confidence_1to1(item1, item2):
    pass
